## Eurostat data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyodbc
import pycountry
pd.set_option('display.max_columns', None)

### Arrivals

In [2]:
df_arrivals = pd.read_excel('./eurostat/TOUR_OCC_ARN2$DEFAULTVIEW1621782256780.xlsx', 'Sheet 5', skiprows=9, nrows=494)

In [3]:
df_arrivals.drop(0, axis=0, inplace=True)
cols = [c for c in df_arrivals.columns if c[:7] == 'Unnamed']
df_arrivals.drop(cols, axis=1, inplace=True)

In [4]:
df_arrivals = df_arrivals.rename(columns = {'TIME' : 'Region'})

In [5]:
df_arrivals.head()

,Region,2015,2016,2017,2018,2019
1,European Union - 27 countries (from 2020),842094674,876845557,928554906,963481864,999794950
2,European Union - 28 countries (2013-2020),966504943,998886631,1059809056,1097315026,1140645513
3,European Union - 27 countries (2007-2013),952347917,983440040,1042399119,1078666089,1121092018
4,"Euro area (EA11-1999, EA12-2001, EA13-2007, EA...",721488022,746546450,789630697,817254588,847478736
5,Belgium,15846926,15210781,16319257,17559915,18057611


### Establishments

In [6]:
df_establishments = pd.read_excel('./eurostat/TOUR_CAP_NUTS2$DEFAULTVIEW1622395990444.xlsx', 'Sheet 1', skiprows=9, nrows=499) # unit of measure - number

In [7]:
df_establishments.drop(0, axis=0, inplace=True)
cols = [c for c in df_establishments.columns if c[:7] == 'Unnamed']
df_establishments.drop(cols, axis=1, inplace=True)
df_establishments.drop('2020', axis=1, inplace=True)

In [8]:
df_establishments = df_establishments.rename(columns = {'TIME' : 'Region'})

In [9]:
df_establishments.head()

,Region,2016,2017,2018,2019
1,European Union - 27 countries (from 2020),524752,571674,600499,617953
2,European Union - 28 countries (2013-2020),609332,663863,697311,717555
3,European Union - 27 countries (2007-2013),526099,565522,589099,603794
4,"Euro area (EA11-1999, EA12-2001, EA13-2007, EA...",401724,432691,450604,461693
5,Belgium,8210,8570,9211,9651


### Air transport

In [10]:
df_airtransport = pd.read_excel('./eurostat/TRAN_R_AVPA_NM$DEFAULTVIEW1622396128371.xlsx', 'Sheet 1', skiprows=8, nrows=353) # unit of measure - thousand

In [11]:
df_airtransport.drop(0, axis=0, inplace=True)
cols = [c for c in df_airtransport.columns if c[:7] == 'Unnamed']
df_airtransport.drop(cols, axis=1, inplace=True)

In [12]:
df_airtransport = df_airtransport.rename(columns = {'TIME' : 'Code', 'TIME.1' : 'Region'})

In [13]:
df_airtransport.head()

,Code,Region,2015,2016,2017,2018,2019
1,BE,Belgium,30955,30114,33259,34506,35384
2,BE2,Vlaams Gewest,23722,22448,25384,26326,27021
3,BE21,Prov. Antwerpen,195,257,255,282,290
4,BE24,Prov. Vlaams-Brabant,23269,21769,24776,25637,26287
5,BE25,Prov. West-Vlaanderen,260,423,354,407,445


### Population

In [14]:
df_population = pd.read_excel('./eurostat/DEMO_R_D2JAN$DEFAULTVIEW1622396054950.xlsx', 'Sheet 1', skiprows=9, nrows=507) # unit of measure - number

In [15]:
df_population.drop(0, axis=0, inplace=True)
cols = [c for c in df_population.columns if c[:7] == 'Unnamed']
df_population.drop(cols, axis=1, inplace=True)

In [16]:
df_population = df_population.rename(columns = {'TIME' : 'Code', 'TIME.1' : 'Region'})

In [17]:
df_population.head()

,Code,Region,2016,2017,2018,2019,2020
1,EU27_2020,European Union - 27 countries (from 2020),444802830,445534430,446208557,446446444,447319916
2,EU28,European Union - 28 countries (2013-2020),510181874,511378572,512372000,513093556,:
3,EU27_2007,European Union - 27 countries (2007-2013),505991205,507224359,508266507,509017310,:
4,BE,Belgium,11311117,11351727,11398589,11455519,11522440
5,BE1,Région de Bruxelles-Capitale/Brussels Hoofdste...,1201285,1199095,1205492,1215290,1223364


### Final df

In [18]:
regions = ["País Vasco",
"Comunidad de Madrid", 
"Cataluña",
"Comunitat Valenciana", 
"Illes Balears", 
"Andalucía", 
"Lombardia",
"Provincia Autonoma di Bolzano/Bozen",
"Provincia Autonoma di Trento",
"Veneto",
"Emilia-Romagna",
"Toscana",
"Lazio",
"Campania",
"Puglia",
"Sicilia",
"Norte", 
"Área Metropolitana de Lisboa" ]

In [19]:
# tylko potrzebne dane (2019) + usunięcie duplikatów
df_arrivals = df_arrivals.loc[df_arrivals.Region.isin(regions), ['Region', '2019']].drop_duplicates()
df_establishments = df_establishments.loc[df_establishments.Region.isin(regions), ['Region', '2019']].drop_duplicates()
df_airtransport = df_airtransport.loc[df_airtransport.Region.isin(regions), ['Code', 'Region', '2019']].drop_duplicates()
df_population = df_population.loc[df_population.Region.isin(regions), ['Code', 'Region', '2019']].drop_duplicates()

In [20]:
# łączenie wszystkich ramek
df_1 = pd.merge(df_arrivals, df_establishments, how='inner', on='Region', suffixes=('_arr', '_est'))
df_2 = pd.merge(df_airtransport, df_population, how='right', on=['Code', 'Region'], suffixes=('_air', '_pop'))
df_final = pd.merge(df_1, df_2, how='left', on='Region')

In [21]:
df_final = df_final.loc[:,['Code', 'Region', '2019_arr', '2019_est', '2019_air', '2019_pop']]
df_final = df_final.query('Code != "ES3"').reset_index(drop=True)

In [22]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Code      18 non-null     object
 1   Region    18 non-null     object
 2   2019_arr  18 non-null     object
 3   2019_est  18 non-null     object
 4   2019_air  16 non-null     object
 5   2019_pop  18 non-null     object
dtypes: object(6)
memory usage: 992.0+ bytes


In [23]:
# zsumowanie danych dla regionu Trentino
df_final.iloc[df_final.Code == 'ITH2', 2:6] = df_final.values[7:9, 2:6].sum(axis=0)
df_final = df_final.drop(7, axis=0).reset_index(drop=True)

In [24]:
# zmiana typów zmiennych
cols = df_final.columns[2:]
for col in cols:
    if col != '2019_air':
        df_final[col] = df_final[col].astype(int)
    if col == '2019_air':
        df_final[col] = df_final[col].astype(float)


In [25]:
# uwzględnienie jednostki w AirTransport
df_final['2019_air'] = df_final['2019_air']*1000

In [26]:
# zmiana nazw kolumn
df_final = df_final.rename(columns = {'Code' : 'RegionCode', '2019_arr' : 'TouristArrivals',
                                        '2019_est' : 'TouristEstablishments', '2019_air' :                                                      'AirPassengers', '2019_pop' : 'Population'})

In [27]:
# dodanie kolumny countrycode i country
df_final['CountryCode'] = df_final.RegionCode.str.slice(0,2)
for i, val in df_final.iterrows():
    country = pycountry.countries.get(alpha_2=df_final.loc[i, 'CountryCode'])
    df_final.loc[i, 'Country'] = country.name

In [28]:
# zmiana na odpowiednie nazwy regionów
df_final.loc[:5, 'Region'] = pd.Series(['Basque Country', 'Comunidad de Madrid', 'Catalonia', 'Valencia', 'Islas Baleares', 'Andalucía'])
df_final.loc[7, 'Region'] = 'Trentino'
df_final.loc[16, 'Region'] = 'Lisbon'

In [29]:
# odpowiednia kolejnosc kolumn
df_final = df_final[['Country', 'CountryCode', 'Region', 'RegionCode', 'TouristArrivals', 'TouristEstablishments', 'AirPassengers', 'Population']]

In [30]:
# zmiana NaN na 0 (tymczasowo w celu odpowiedniego wczytania do bazy)
df_final.loc[df_final.AirPassengers.isna(), 'AirPassengers'] = 0

In [31]:
df_final

,Country,CountryCode,Region,RegionCode,TouristArrivals,TouristEstablishments,AirPassengers,Population
0,Spain,ES,Basque Country,ES21,3866136,1232,6353000.0,2177880
1,Spain,ES,Comunidad de Madrid,ES30,13774677,1929,59747000.0,6641649
2,Spain,ES,Catalonia,ES51,26155473,6698,54693000.0,7566431
3,Spain,ES,Valencia,ES52,12363818,3990,23401000.0,4974969
4,Spain,ES,Islas Baleares,ES53,12425741,2772,40285000.0,1188220
5,Spain,ES,Andalucía,ES61,24202798,8403,30414000.0,8427405
6,Italy,IT,Lombardia,ITC4,17509461,12002,49096000.0,10010833
7,Italy,IT,Trentino,ITH2,12222269,13900,0.0,1074034
8,Italy,IT,Veneto,ITH3,20194655,62461,18404000.0,4884590
9,Italy,IT,Emilia-Romagna,ITH5,11597928,16592,9905000.0,4459453


### Loading to database

In [32]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-QEE21EV;'
                      'Database=Airbnb_eurostat;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [33]:
cursor.execute('CREATE TABLE RegionalStatistics ( \
               Country varchar(50),\
               CountryCode varchar(2),\
               Region varchar(100),\
               RegionCode varchar(4),\
               TouristArrivals int,\
               TouristEstablishments int,\
               AirPassengers int,\
               Population int)')
conn.commit()

In [34]:
for row in df_final.itertuples():
    cursor.execute('''
                INSERT INTO RegionalStatistics
                VALUES (?,?,?,?,?,?,?,?)
                ''',
                 row.Country, row.CountryCode, 
                 row.Region, row.RegionCode, 
                 row.TouristArrivals, row.TouristEstablishments, 
                 row.AirPassengers, row.Population)
conn.commit()

In [35]:
# zmiana 0 na własciwa wartość (NULL)
cursor.execute('''
               UPDATE Airbnb_eurostat.dbo.RegionalStatistics
               SET AirPassengers = NULL
               WHERE RegionCode = 'ITH2'
               ''')
conn.commit()

In [36]:
df_final.to_csv('eurostat_data.csv', index=False, header=True)